# README
This notebook fetches the CBOE delayed quotes from https://www.cboe.com/delayed_quotes/spx/quote_table . We clean the data and calculate some new columns before appending to the existing parquet file that is storing the data historically. Note that this file can only be up to 20GB but that should be fine given the small size of parquet files.

# Imports

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
import os

# Helper Functions

In [2]:
def is_third_friday(d):
    """
    Function to check if the provided date is the third friday of the month (monthly expiry)
    """
    return d.weekday() == 4 and 15 <= d.day <= 21

# Fetch Data

In [3]:
# Symbol
symbol = '_SPX'

# Endpoint
endpoint = f'https://cdn.cboe.com/api/global/delayed_quotes/options/{symbol}.json'

In [4]:
def fetch_latest_data(
    endpoint: str
) -> None: # TODO CHANGE RETURN TYPE
    """
    Function to fetch the latest data, handling exceptions
    """
    try:
        # Make call
        response = requests.get(endpoint)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Convert data to json
            data = response.json()
            
            # Check we have options data and index price, read into dataframe if so.
            options_data = data['data']['options']
            index_price = data['data']['current_price']
            if not (options_data and index_price):
                print("No data available for today.")
                return pd.DataFrame()
            else:
                df = pd.DataFrame(options_data)
                df['index_price'] = index_price
                return df
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return pd.DataFrame()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
df = fetch_latest_data(
    endpoint=endpoint
)

In [6]:
df

,option,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,gamma,...,change,open,high,low,tick,last_trade_price,last_trade_time,percent_change,prev_day_close,index_price
0,SPX241220C00200000,5804.6,64.0,5817.70,5.0,4.5906,5282.0,0.0,1.0000,0.0000,...,0.0,0.0,0.0,0.0,up,5806.32,2024-11-26T12:38:57,0.0,5816.250000,6013.04
1,SPX241220P00200000,0.0,0.0,0.05,214.0,3.7136,11724.0,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,up,0.05,2024-10-07T14:33:55,0.0,0.025000,6013.04
2,SPX241220C00400000,5600.5,5.0,5618.40,5.0,0.0000,2857.0,0.0,1.0000,0.0000,...,0.0,0.0,0.0,0.0,up,5490.92,2024-11-20T12:06:25,0.0,5617.050049,6013.04
3,SPX241220P00400000,0.0,0.0,0.05,212.0,2.9304,5783.0,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,no_change,0.05,2024-08-07T14:01:24,0.0,0.025000,6013.04
4,SPX241220C00500000,5500.7,10.0,5518.80,10.0,0.0000,14.0,0.0,1.0000,0.0000,...,0.0,0.0,0.0,0.0,up,5467.00,2024-11-14T13:21:02,0.0,5517.299805,6013.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23827,SPXW250930P07500000,1221.7,1.0,1254.80,1.0,0.1144,0.0,0.0,-0.9649,0.0001,...,0.0,0.0,0.0,0.0,no_change,0.00,None,0.0,1230.150024,6013.04
23828,SPXW250930C07600000,7.1,140.0,7.50,72.0,0.1139,94.0,0.0,0.0323,0.0001,...,0.0,0.0,0.0,0.0,down,7.50,2024-11-25T12:34:20,0.0,7.400000,6013.04
23829,SPXW250930P07600000,1315.7,1.0,1348.30,1.0,0.1166,2.0,0.0,-0.9734,0.0001,...,0.0,0.0,0.0,0.0,down,1436.63,2024-11-18T12:35:34,0.0,1323.700012,6013.04
23830,SPXW250930C07800000,4.4,158.0,4.70,14.0,0.1171,115.0,0.0,0.0210,0.0001,...,0.0,0.0,0.0,0.0,no_change,4.40,2024-11-26T11:55:50,0.0,4.600000,6013.04


# Clean/Transform Data

In [7]:
def add_columns(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Function to add columns that will be stored
    """
    # Return empty frame if input is empty
    if df.empty:
        return df

    # Copy of frame to not amend original
    new_df = df.copy()

    # Add date column and make it first column
    new_df['asof_date'] = datetime.today().date().strftime(format="%Y-%m-%d")
    columns = ['asof_date'] + [col for col in new_df.columns if col != 'asof_date']
    new_df = new_df[columns]
    
    # Add expiry, type (call/put) and strike. Then sort by expiry
    without_w = new_df['option'].str.replace("W", "")
    new_df['expiry'] = pd.to_datetime(without_w.str[3:9], format="%y%m%d")
    new_df['type'] = without_w.str[9]
    new_df['strike'] = without_w.str[11:-3].astype(int)
    new_df = new_df.sort_values(by='expiry').reset_index(drop=True)

    return new_df

In [8]:
df = add_columns(df=df)

In [9]:
df

,asof_date,option,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,...,low,tick,last_trade_price,last_trade_time,percent_change,prev_day_close,index_price,expiry,type,strike
0,2024-11-27,SPXW241127C02400000,3609.9,7.0,3617.2,7.0,0.0000,0.0,0.0,1.0000,...,0.00,up,3597.40,2024-11-26T10:24:41,0.0000,3619.300049,6013.04,2024-11-27,C,2400
1,2024-11-27,SPXW241127P06010000,7.3,64.0,7.5,86.0,0.1448,1283.0,5855.0,-0.4373,...,5.70,up,7.40,2024-11-27T10:03:19,-12.9412,8.500000,6013.04,2024-11-27,P,6010
2,2024-11-27,SPXW241127C06010000,10.8,32.0,11.0,14.0,0.1446,1328.0,3322.0,0.5627,...,9.77,no_change,10.70,2024-11-27T10:03:16,-42.4731,18.600000,6013.04,2024-11-27,C,6010
3,2024-11-27,SPXW241127P06005000,5.6,77.0,5.7,9.0,0.1487,1073.0,3862.0,-0.3562,...,4.50,no_change,5.80,2024-11-27T10:03:17,-17.1429,7.000000,6013.04,2024-11-27,P,6005
4,2024-11-27,SPXW241127C06005000,14.1,1.0,14.3,26.0,0.1494,1259.0,1071.0,0.6438,...,12.70,no_change,14.30,2024-11-27T10:03:14,-35.2941,22.100000,6013.04,2024-11-27,C,6005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23827,2024-11-27,SPX291221P02400000,35.1,14.0,40.9,14.0,0.2876,23.0,0.0,-0.0223,...,0.00,up,40.43,2024-11-25T14:16:07,0.0000,37.650000,6013.04,2029-12-21,P,2400
23828,2024-11-27,SPX291221C02400000,3871.2,5.0,3951.2,5.0,0.2846,0.0,0.0,1.0000,...,0.00,no_change,0.00,None,0.0000,0.000000,6013.04,2029-12-21,C,2400
23829,2024-11-27,SPX291221P02200000,28.6,14.0,33.2,14.0,0.2969,28.0,0.0,-0.0180,...,0.00,down,29.50,2024-11-26T12:20:26,0.0000,30.550001,6013.04,2029-12-21,P,2200
23830,2024-11-27,SPX291221P02000000,22.5,13.0,26.8,13.0,0.3067,765.0,0.0,-0.0143,...,0.00,down,24.49,2024-11-25T14:17:28,0.0000,24.450000,6013.04,2029-12-21,P,2000


# Store Data

In [10]:
# Path to file
file_path = "/kaggle/working/spx_options_delayed_quotes.parquet"

In [11]:
# Check if the Parquet file already exists 
if os.path.exists(file_path): 
    # Read the existing data 
    existing_df = pd.read_parquet(file_path) 
    
    # Append the new data 
    combined_df = pd.concat([existing_df, df], ignore_index=True) 
else: 
    # If the file does not exist, the combined DataFrame is just the new data 
    combined_df = df

# Write the combined DataFrame back to the Parquet file 
combined_df.to_parquet(file_path, index=False) 